In [1]:
import numpy as np
import pickle

        phrase2 = token.split("’")


In [8]:
vocabs = np.loadtxt("tokenized_vocabs_no_utf.txt", dtype='<U92',encoding="utf-8")

In [10]:
def is_ascii(s):
    return all(ord(c) < 128 for c in s)

In [11]:
def is_alpha(s):
    return s.isalpha()

In [22]:
real_ascii_vocab = []
for vocab in vocabs:
    if(is_ascii(vocab) is False or is_alpha(vocab) is False):
        continue
    real_ascii_vocab.append(vocab)
    

In [23]:
real_ascii_vocab_num = []
for vocab in vocabs:
    if(is_ascii(vocab) is False):
        continue
    real_ascii_vocab_num.append(vocab)

In [28]:
real_ascii_vocab.append("\'")

In [25]:
len(real_ascii_vocab_num)

698114

In [29]:
len(real_ascii_vocab)

695347

In [30]:
import os
import numpy as np
import networkx as nx
from collections import Counter
import node2vec
import pandas as pd

In [31]:
from gensim.models import Word2Vec

In [32]:
train_dataset = pd.read_csv("./conceptnet_modify_update.csv", index_col=0, encoding='utf-8')

c:\users\qkrwo\anaconda3\envs\node2vec\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [33]:
def add_nodes(G, df, col, type_name):
    """Add entities to G from the 'col' column of the 'df' DataFrame. The new nodes are annotated with 'type_name' label."""
    nodes = list(df[~df[col].isnull()][col].unique())
    G.add_nodes_from([(n,dict(type=type_name)) for n in nodes])
    print("Nodes (%s,%s) were added" % (col, type_name))
    
def add_nodes_2col(G, df, col1, col2, type_name):
    """Add entities to G from the 'col' column of the 'df' DataFrame. The new nodes are annotated with 'type_name' label."""
    nodes = list((df[~df[col1].isnull()][col1] + df[~df[col2].isnull()][col2]).unique())
    print(len(nodes))
    G.add_nodes_from([(n,dict(type=type_name)) for n in nodes])
    print("Nodes (%s,%s) were added" % (col1, type_name))
    print("Nodes (%s,%s) were added" % (col2, type_name))
    print(word_nodes)
    
def add_links(G, df, col1, col2, type_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[(~df[col1].isnull()) & (~df[col2].isnull())]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    G.add_edges_from([(src, trg, dict(type=type_name)) for src, trg in links])
    print("Edges (%s->%s,%s) were added" % (col1, col2, type_name))
    
def add_relation(G, df, col1, col2, relation_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[~(df['relation'] != relation_name)]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    G.add_edges_from([(src, trg, dict(type=relation_name)) for src, trg in links])
    print("Edges (%s->%s,%s) were added" % (col1, col2, relation_name))

    
def add_relation_words(G, df, col1, col2, relation_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[~(df['relation'] != relation_name)]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    word_links = []
    rel_links = []
    for src, trg in links:
        src, trg = str(src), str(trg)
        src_words = src.split("_")
        trg_words = trg.split("_")
        for i in range(len(src_words) - 1) :
            word_links.append((src_words[i], src_words[i+1], dict(type="sub_continous")))
        rel_links.append((src_words[-1], trg_words[0], dict(type=relation_name)))
        for i in range(len(trg_words) - 1) :
            word_links.append((trg_words[i], trg_words[i+1], dict(type="val_continous")))
    G.add_edges_from(word_links)
    G.add_edges_from(rel_links)
    print("relation number = ", len(rel_links))
    print("Edges (%s->%s,%s) were added" % (col1, col2, relation_name))
    
def add_relation_words2(G, df, col1, col2, relation_name):
    """Add links to G from the 'df' DataFrame. The new edges are annotated with 'type_name' label."""
    df_tmp = df[~(df['relation'] != relation_name)]
    links = list(zip(df_tmp[col1],df_tmp[col2]))
    word_links = []
    rel_links = []
    for src, trg in links:
        src, trg = str(src), str(trg)
        rel_links.append((src.split("/")[0], trg.split("/")[0], dict(type=relation_name)))
    G.add_edges_from(rel_links)
    print("Edges (%s->%s,%s) were added %s" % (col1, col2, relation_name, len(rel_links)))

In [34]:
G = nx.DiGraph()